Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Data Preparation

In this notebook, we use a subset of [Stack Exchange network](https://archive.org/details/stackexchange) question data which includes original questions tagged as 'JavaScript', their duplicate questions and their answers. Here, we provide the steps to prepare the data to use in model development for training a model that will match a new question with an existing original question. 

In [ ]:
import os
import pandas as pd
from utilities import read_csv_gz, clean_text, round_sample_strat, random_merge

Below, we define some parameters that will be used in the data cleaning as well as train and test set preparation.

In [ ]:
# The size of the test set
test_size = 0.21
# The minimum length of clean text
min_text = 150
# The minimum number of duplicates per question
min_dupes = 12
# The maximum number of duplicate matches
match = 20
# The output files path
outputs_path = os.path.join('.', 'data_folder')

## Data cleaning

Next, we download the questions, duplicate questions and answers and load the datasets into pandas dataframes using the helper functions.

In [ ]:
# URLs to original questions, duplicate questions, and answers.
data_url = "https://bostondata.blob.core.windows.net/stackoverflow/{}"
questions_url = data_url.format("orig-q.tsv.gz")
dupes_url = data_url.format("dup-q.tsv.gz")
answers_url = data_url.format("ans.tsv.gz")

In [ ]:
# Load datasets.
questions = read_csv_gz(questions_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
dupes = read_csv_gz(dupes_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
answers = read_csv_gz(answers_url, names=('Id', 'Text0'))

Let's now check the dataframes. Notice that questions and duplicates have "AnswerID" column that would help match with the index of answers dataframe.

In [ ]:
questions.head()

In [ ]:
dupes.head()

In [ ]:
answers.head()

Let's check the first original question's text.

In [ ]:
questions.iloc[0,1]

Let's now check the duplicates for that question.

In [ ]:
dupes[dupes.AnswerId == questions.iloc[0, 0]]

Below is the answer to the original question.

In [ ]:
answers.at[questions.iloc[0,0],'Text0']

Next, we use the helper functions to clean questions, duplicates and answers from unwanted text such as code, html tags and links. Notice that we add a new column 'Text' to each dataframe for clean text in lowercase.

In [ ]:
# Clean up all text, and keep only data with some clean text.
for df in (questions, dupes, answers):
    df["Text"] = df.Text0.apply(clean_text).str.lower()

In [ ]:
questions = questions[questions.Text.str.len() > 0]
answers = answers[answers.Text.str.len() > 0]
dupes = dupes[dupes.Text.str.len() > 0]

Let's compare the first original question and cleaned version as an example.

In [ ]:
# Original question.
questions.iloc[0,1]

In [ ]:
# After cleaning.
questions.iloc[0,3]

it turns out that some duplicate questions were also in original questions. Also, some original questions and some duplicate questions were duplicated in the datasets. In the following, we remove them from the dataframes.

In [ ]:
# First, remove dupes that are questions, then remove duplicated questions and dupes.
dupes = dupes[~dupes.index.isin(questions.index)]
questions = questions[~questions.index.duplicated(keep='first')]
dupes = dupes[~dupes.index.duplicated(keep='first')]

We also make sure we keep questions with answers and duplicates.

In [ ]:
# Keep only questions with answers and dupes, answers to questions, and dupes of questions.
questions = questions[
    questions.AnswerId.isin(answers.index) & questions.AnswerId.isin(dupes.AnswerId)
]
answers = answers[answers.index.isin(questions.AnswerId)]
dupes = dupes[dupes.AnswerId.isin(questions.AnswerId)]

In [ ]:
# Verify data integrity.
assert questions.AnswerId.isin(answers.index).all()
assert answers.index.isin(questions.AnswerId).all()
assert questions.AnswerId.isin(dupes.AnswerId).all()
assert dupes.AnswerId.isin(questions.AnswerId).all()

Below are some statistics on the data. Notice that some questions have very low number of duplicates while others may have a large number. 

In [ ]:
# Report on the data.
print("Text statistics:")
print(
    pd.DataFrame(
        [
            questions.Text.str.len().describe().rename("questions"),
            answers.Text.str.len().describe().rename("answers"),
            dupes.Text.str.len().describe().rename("dupes"),
        ]
    )
)
print("\nDuplication statistics:")
print(pd.DataFrame([dupes.AnswerId.value_counts().describe().rename("duplications")]))
print(
    "\nLargest class: {:.2%}".format(
        dupes.AnswerId.value_counts().max() / dupes.shape[0]
    )
)

Now, we reset all indexes to use them as columns in the rest of the steps.

In [ ]:
# Reset each dataframe's index.
questions.reset_index(inplace=True)
answers.reset_index(inplace=True)
dupes.reset_index(inplace=True)

We filter the questions and duplicates to have at least min_text number of characters.

In [ ]:
# Apply the minimum text length to questions and dupes.
questions = questions[questions.Text.str.len() >= min_text]
dupes = dupes[dupes.Text.str.len() >= min_text]

In [ ]:
# Keep only questions with dupes, and dupes of questions.
label_column = "AnswerId"
questions = questions[questions[label_column].isin(dupes[label_column])]
dupes = dupes[dupes[label_column].isin(questions[label_column])]

Here, we remove questions and their duplicates that are less than min_dupes parameter.

In [ ]:
# Restrict the questions to those with a minimum number of dupes.
answerid_count = dupes.groupby(label_column)[label_column].count()
answerid_min = answerid_count.index[answerid_count >= min_dupes]
questions = questions[questions[label_column].isin(answerid_min)]
dupes = dupes[dupes[label_column].isin(answerid_min)]

In [ ]:
 # Verify data integrity.
assert questions[label_column].isin(dupes[label_column]).all()
assert dupes[label_column].isin(questions[label_column]).all()

Here are some statistics on the resulting dataset.

In [ ]:
# Report on the data.
print("Restrictions: min_text={}, min_dupes={}".format(min_text, min_dupes))
print("Restricted text statistics:")
print(
    pd.DataFrame(
        [
            questions.Text.str.len().describe().rename("questions"),
            dupes.Text.str.len().describe().rename("dupes"),
        ]
    )
)
print("\nRestricted duplication statistics:")
print(
    pd.DataFrame([dupes[label_column].value_counts().describe().rename("duplications")])
)
print(
    "\nRestricted largest class: {:.2%}".format(
        dupes[label_column].value_counts().max() / dupes.shape[0]
    )
)

## Prepare train and test sets

In this part, we prepare train and test sets. For training a binary classification model, we will need to construct match and non-match pairs from duplicates and their questions. Finding matching pairs can be accomplished by joining each duplicate with its question. However, non-match examples need to be constructed randomly. 

As a first step, to make sure we train and test the performance of the model on each question, we will need to have examples of match and non-match pairs for each question both in train and test sets. In order to achieve that, we split the duplicates in a stratified manner into train and test sets making sure at least 1 or more duplicates per question is in the test set depending on test_size parameter and number of duplicates per each question.

In [ ]:
# Split dupes into train and test ensuring at least one of each label class is in test.
dupes_test = round_sample_strat(dupes, dupes[label_column], frac=test_size)
dupes_train = dupes[~dupes.Id.isin(dupes_test.Id)]

In [ ]:
assert (dupes_test[label_column].unique().shape[0] == dupes[label_column].unique().shape[0])

In [ ]:
# The relevant columns for text pairs data.
balanced_pairs_columns = ['Id_x', 'AnswerId_x', 'Text_x', 'Id_y', 'Text_y', 'AnswerId_y', 'Label', 'n']

Next, we pair each training duplicate in train set with its matching question and N-1 random questions using the helper function.

In [ ]:
# Use AnswerId to pair each training dupe with its matching question and also with N-1 questions not its match.
%time balanced_pairs_train = random_merge(dupes_train, questions, N=match)

Labeling is done such that matching pairs are labeled as 1 and non-match pairs are labeled as 0.

In [ ]:
# Label records by matching AnswerIds.
balanced_pairs_train["Label"] = (
    balanced_pairs_train.AnswerId_x == balanced_pairs_train.AnswerId_y
).astype(int)

In [ ]:
# Keep only the relevant data.
balanced_pairs_train = balanced_pairs_train[balanced_pairs_columns]

In [ ]:
balanced_pairs_train.head()

In [ ]:
# Sort the data by dupe ID and Label.
balanced_pairs_train.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)

In testing set, we match each duplicate with all the original questions and label them same way as training set.

In [ ]:
# Use AnswerId to pair each testing dupe with all questions.
%time balanced_pairs_test = random_merge(dupes_test, questions, N=questions.shape[0])

In [ ]:
# Label records by matching AnswerIds.
balanced_pairs_test["Label"] = (
    balanced_pairs_test.AnswerId_x == balanced_pairs_test.AnswerId_y
).astype(int)

In [ ]:
# Keep only the relevant data.
balanced_pairs_test = balanced_pairs_test[balanced_pairs_columns]

In [ ]:
balanced_pairs_test.head()

In [ ]:
# Sort the data by dupe ID and Label.
balanced_pairs_test.sort_values(
    by=["Id_x", "Label"], ascending=[True, False], inplace=True
)

Finally, we report the final train and test sets and save as text files to be used by modeling.

In [ ]:
# Report on the datasets.
print(
    "balanced_pairs_train: {:,} rows with {:.2%} matches".format(
        balanced_pairs_train.shape[0], balanced_pairs_train.Label.mean()
    )
)
print(
    "balanced_pairs_test: {:,} rows with {:.2%} matches".format(
        balanced_pairs_test.shape[0], balanced_pairs_test.Label.mean()
    )
)

In [ ]:
os.makedirs(outputs_path, exist_ok=True)

# Save the data.
balanced_pairs_train_path = os.path.join(outputs_path, "balanced_pairs_train.tsv")
print(
    "Writing {:,} to {}".format(
        balanced_pairs_train.shape[0], balanced_pairs_train_path
    )
)
balanced_pairs_train.to_csv(
    balanced_pairs_train_path, sep="\t", header=True, index=False
)

balanced_pairs_test_path = os.path.join(outputs_path, "balanced_pairs_test.tsv")
print(
    "Writing {:,} to {}".format(balanced_pairs_test.shape[0], balanced_pairs_test_path)
)
balanced_pairs_test.to_csv(balanced_pairs_test_path, sep="\t", header=True, index=False)

# Save original questions to be used for scoring later.
questions_path = os.path.join(outputs_path, "questions.tsv")
print("Writing {:,} to {}".format(questions.shape[0], questions_path))
questions.to_csv(questions_path, sep="\t", header=True, index=False)

# Save the test duplicate questions to be used with the scoring function.
dupes_test_path = os.path.join(outputs_path, "dupes_test.tsv")
print("Writing {:,} to {}".format(dupes_test.shape[0], dupes_test_path))
dupes_test.to_csv(dupes_test_path, sep="\t", header=True, index=False)

We can now move on to [train on local](02_TrainOnLocal.ipynb) notebook to train our model using Azure Machine Learning.